<a href="https://colab.research.google.com/github/heartdbwls/robot-arm/blob/main/skeleton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json
from PIL import Image, ImageDraw, ImageFont

In [2]:
from google.colab import files

uploaded = files.upload()

Saving project.json to project.json


In [3]:
#이미지 폴더 경로
image_folder_path = 'image_folder'

# 경로 내의 이미지 파일 목록 가져오기
import os
image_files = os.listdir(image_folder_path)

In [4]:
#이미지 크기
new_image_width =8000
new_image_height =6000

In [5]:
#점의 크기
dot_size = 20

In [6]:
#선의 두께
line_width=4

In [7]:
#라벨 폰트 크기
font_size=90

In [8]:
#라벨 색상
label_colors = [
    (255,255,255), #J1
    (255,255,255), #J2
    (255,255,255), #J3
    (0,0,0), #J4
    (255,255,255), #J5
    (255,255,255), #J6
    (255,255,255), #J7 (Flange)
]

In [9]:
#점 색상
dot_colors = [
    (255,0,0), #J1
    (0,255,0), #J2
    (0,0,255), #J3
    (255,255,0), #J4
    (255,0,255), #J5
    (0,255,255), #J6
    (255,165,0), #J7(Flange)
]

In [10]:
#JSON 파일 열기
import io

json_file_path = list(uploaded.keys())[0]  # 업로드된 파일의 가상 경로
with io.open(json_file_path, "r") as f:
    data = json.load(f)

In [11]:
!apt-get install fonts-nanum -qq

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 폰트 경로 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=font_size)


Selecting previously unselected package fonts-nanum.
(Reading database ... 122532 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20180306-3_all.deb ...
Unpacking fonts-nanum (20180306-3) ...
Setting up fonts-nanum (20180306-3) ...
Processing triggers for fontconfig (2.13.1-2ubuntu3) ...


In [12]:
#JSON 파일 수정
def convert_format(json_data):
    new_json_data = []
    for item in json_data:
        new_item = {
            "file_upload": item["file_upload"],
            "id": item["id"],
            "annotations": []
        }
        for annotation in item["annotations"]:
            new_annotation = {
                "id": annotation["id"],
                "completed_by": annotation["completed_by"],
                "result": []
            }
            for result in annotation["result"]:
                value = {
                    "x": result["value"].get("x") or 0,  # 라벨값이 없을 때 0으로 설정
                    "y": result["value"].get("y") or 0,  # 라벨값이 없을 때 0으로 설정
                    "width": result["value"].get("width") or 0,  # 라벨값이 없을 때 0으로 설정
                    "keypointlabels": result["value"].get("keypointlabels", [])
                }
                new_result = {
                    "original_width": result["original_width"],
                    "original_height": result["original_height"],
                    "image_rotation": result["image_rotation"],
                    "value": value,
                    "id": result["id"],
                    "from_name": result["from_name"],
                    "to_name": result["to_name"],
                    "type": result["type"],
                    "origin": result["origin"]
                }
                new_annotation["result"].append(new_result)
            new_item["annotations"].append(new_annotation)
        new_json_data.append(new_item)
    return new_json_data

# 첫 번째 파일 경로
first_file_path = r'project.json'

# 첫 번째 파일 읽기
with open(first_file_path, "r") as file:
    first_file_content = file.read()

# JSON 파싱
first_file_data = json.loads(first_file_content)

# 형식 변환
new_json_data = convert_format(first_file_data)

# 두 번째 파일 경로
output_directory = r'/content/'
second_file_path = os.path.join(output_directory, 'project.json')

# 두 번째 파일 쓰기
with open(second_file_path, "w") as file:
    json.dump(new_json_data, file)

print("파일이 생성되었습니다.")

파일이 생성되었습니다.


In [13]:
#id당 keypointlabel정리
def get_valid_keypoints(json_data):
    valid_keypoints_dict = {}
    for item in json_data:
        id_value = item["id"]
        valid_keypoints = []
        for annotation in item["annotations"]:
            for result in annotation["result"]:
                keypointlabels = result["value"].get("keypointlabels", [])
                if keypointlabels:
                    valid_keypoints.extend(keypointlabels)
        valid_keypoints_dict[id_value] = valid_keypoints
    return valid_keypoints_dict

#유용하게 활용하기 위해 valid_keypoints_dict 안의 J제거
def remove_J_from_keypoints(valid_keypoints_dict):
    new_keypoints_dict = {}
    for key, value in valid_keypoints_dict.items():
        new_value = [int(item[1:]) for item in value]
        new_keypoints_dict[key] = new_value
    return new_keypoints_dict

# 유효한 keypoint들을 담은 딕셔너리 생성
valid_keypoints_dict = get_valid_keypoints(new_json_data)

# J 제거한 keypoint 딕셔너리 생성
new_keypoints_dict = remove_J_from_keypoints(valid_keypoints_dict)

# 1부터 7까지의 숫자에 대응하는 위치에 값 넣기
for key, value in new_keypoints_dict.items():
    new_value = [0] * 7
    for item in value:
        if 1 <= item <= 7:
            new_value[item-1] = item
    new_keypoints_dict[key] = new_value

print(new_keypoints_dict)

{1: [1, 2, 3, 4, 5, 6, 7], 2: [1, 2, 3, 4, 5, 6, 7], 3: [1, 2, 3, 0, 5, 6, 7], 4: [0, 0, 0, 0, 0, 0, 7], 5: [1, 2, 3, 4, 5, 6, 7], 6: [1, 2, 3, 4, 5, 6, 7], 7: [1, 2, 3, 4, 5, 6, 7], 8: [1, 2, 3, 4, 5, 6, 7], 9: [1, 2, 3, 4, 5, 6, 7], 10: [1, 2, 3, 4, 5, 6, 7], 11: [1, 2, 3, 4, 5, 6, 7], 12: [1, 2, 3, 4, 5, 6, 7], 13: [1, 2, 3, 4, 5, 6, 7], 14: [1, 2, 3, 4, 5, 6, 7], 15: [1, 2, 3, 4, 5, 6, 7], 16: [1, 2, 3, 4, 5, 6, 7], 17: [1, 0, 0, 0, 5, 0, 7], 18: [1, 2, 3, 4, 5, 6, 7], 19: [1, 2, 3, 0, 0, 6, 7], 20: [1, 2, 3, 4, 5, 6, 7], 21: [1, 2, 3, 4, 5, 6, 7], 22: [1, 2, 3, 4, 5, 6, 7], 23: [1, 2, 3, 4, 5, 6, 7], 24: [1, 2, 3, 4, 5, 6, 7], 25: [1, 2, 3, 4, 5, 6, 7], 26: [1, 2, 3, 4, 5, 6, 7], 27: [1, 2, 3, 4, 5, 6, 7], 28: [1, 2, 3, 4, 5, 6, 7], 29: [1, 2, 3, 4, 5, 6, 7], 30: [1, 2, 3, 4, 5, 6, 7], 31: [1, 2, 3, 4, 5, 6, 7], 32: [1, 2, 3, 4, 5, 6, 7], 33: [1, 2, 3, 4, 5, 6, 7], 34: [1, 2, 3, 4, 5, 6, 7], 35: [1, 2, 3, 4, 5, 6, 7], 36: [1, 2, 3, 4, 5, 6, 7], 37: [1, 2, 3, 4, 5, 6, 7], 38: [1, 2

In [19]:
# 결과물을 저장할 폴더명
output_folder = 'output_folder'

# 폴더 생성
os.makedirs(output_folder, exist_ok=True)

In [20]:
for entry in new_json_data:
    # 이미지 파일 이름 추출
    image_filename = entry["file_upload"]

    # 이미지 제목 추출
    image_title = image_filename.split(".")[0][9:]

    # 이미지 파일 경로 생성
    image_path = os.path.join('image_folder', image_title + ".JPG")

    # 이미지 파일 확인
    if not os.path.exists(image_path):
        print("Image file not found:", image_path)
        continue

    # 이미지 열기
    image = Image.open(image_path)

    # 이미지 크기 조정
    image = image.resize((new_image_width, new_image_height))

    # 키포인트 데이터 가져오기
    keypoints = entry["annotations"][0]["result"]

    # 유용한 키포인트
    valid_keypoints = [keypoint for keypoint in keypoints if keypoint["value"]["x"]]

    # 유효한 키포인트가 없으면 생략
    for i in range(1, len(new_keypoints_dict) + 1):
        if i not in new_keypoints_dict or len(new_keypoints_dict[i]) == 0:
            continue

        # 키포인트 인덱스 정보 가져오기
        keypoint_indices = new_keypoints_dict[i]

        # 키포인트 위치 및 스켈레톤 정보 가져오기
        keypoint_locations = [valid_keypoints[j - 1] for j in keypoint_indices if 0 < j <= len(valid_keypoints)]
        skeleton_indices = [(j, j + 1) for j in range(len(keypoint_indices) - 1) if keypoint_indices[j] != 0 and keypoint_indices[j + 1] != 0 and 0 < j < len(keypoint_locations)]

        # 이미지에 점 그리기
        draw = ImageDraw.Draw(image)
        for j, keypoint in enumerate(keypoint_locations):
            x_ratio = keypoint["value"]["x"]
            y_ratio = keypoint["value"]["y"]
            x = int(x_ratio * new_image_width / 100)
            y = int(y_ratio * new_image_height / 100)
            label = "J" + str(keypoint_indices[j])
            dot_color = dot_colors[j % len(dot_colors)]
            draw.ellipse((x - dot_size, y - dot_size, x + dot_size, y + dot_size), fill=dot_color, outline=dot_color)
            font = ImageFont.truetype("NanumGothic.ttf", font_size)
            text_width, text_height = draw.textsize(label, font=font)
            label_color = label_colors[j % len(label_colors)]
            draw.rectangle((x + dot_size, y - text_height, x + dot_size + text_width, y), fill=dot_color, outline=dot_color)
            draw.text((x + dot_size, y - text_height), label, fill=label_color, font=font)

        # Skeleton 그리기
        if len(skeleton_indices) > 0:
            for skeleton_index in skeleton_indices:
                keypoint1 = keypoint_locations[skeleton_index[0] - 1]
                keypoint2 = keypoint_locations[skeleton_index[1] - 1]
                x1_ratio = keypoint1["value"]["x"]
                y1_ratio = keypoint1["value"]["y"]
                x2_ratio = keypoint2["value"]["x"]
                y2_ratio = keypoint2["value"]["y"]
                x1 = int(x1_ratio * new_image_width / 100)
                y1 = int(y1_ratio * new_image_height / 100)
                x2 = int(x2_ratio * new_image_width / 100)
                y2 = int(y2_ratio * new_image_height / 100)
                skeleton_color = (255, 0, 0)  # 뼈대 색상 (빨간색)
                draw.line((x1, y1, x2, y2), fill=skeleton_color, width=line_width)

    # 결과물 저장 경로 생성
    output_path = os.path.join(output_folder, image_title + "_labeled.jpg")

    # 결과물 저장
    image.save(output_path)

print("Labeling complete!")



Labeling complete!


In [ ]:
from google.colab import files

folder_path = 'output_folder'

file_list = os.listdir(folder_path)

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path):  # 파일인 경우에만 다운로드
        files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
import shutil

folder_path = 'output_folder'
shutil.rmtree(folder_path)